<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulaci-n-2/blob/main/Venta_de_boletos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VENTA DE BOLETOS TICKETMASTER (FILA VIRTUAL)

Se simula la llegada de clientes a la fila virtual para la compra de boletos en Ticketmaster.

Cada cliente llega en un tiempo aleatorio (siguiendo una distribución exponencial) y se registra el momento de su llegada.

In [2]:
!pip install simpy

In [3]:

import matplotlib.pyplot as plt
import numpy as np
from random import random, seed
import pandas as pd
import simpy

In [21]:
def venta_boletos(env, fila_virtual, num_clientes, texto=True):
    global tasa_llegada, cifras, duracion_compra, tiempos_llegada, tiempos_compra, tiempos_salida

    for i in range(num_clientes):
        #Llegada del cliente a la fila virtual
        r=random()
        tiempo_llegada=-np.log(r)/tasa_llegada
        yield env.timeout(tiempo_llegada)
        tiempos_llegada.append(env.now)

        if texto:
            print(f'\n----El cliente número {i} ha llegado a la fila virtual en el minuto {round(env.now, cifras)}----')

        #Iniciar el proceso de compra para el cliente
        env.process(compra_boletos(env, fila_virtual, i, texto))

Se sigue con el proceso de compra de boletos: el cliente espera su turno en la fila virtual, realiza la compra en un tiempo aleatorio y se registra el momento de inicio y fin de dicha compra.

In [22]:
def compra_boletos(env, fila_virtual, num_cliente, texto=True):
    global duracion_compra, tiempos_compra, tiempos_salida, tiempo_compra_min, tiempo_compra_max

    #Solicitud para iniciar la compra de boletos
    with fila_virtual.request() as solicitud:
        yield solicitud

        #Se genera el tiempo de compra
        r=random()
        tiempo_compra=tiempo_compra_min+(tiempo_compra_max-tiempo_compra_min) * r
        duracion_compra+=tiempo_compra
        tiempos_compra.append(tiempo_compra)

        if texto:
            print(f'****El cliente {num_cliente} ha comenzado la compra en el minuto {round(env.now, cifras)}****')

        #Tiempo en el sistema durante la compra de boletos
        yield env.timeout(tiempo_compra)

        #Registro del momento en que termina la compra
        tiempos_salida.append(env.now)

        if texto:
            print(f'xxxEl cliente {num_cliente} ha terminado la compra en el minuto {round(env.now, cifras)}xxx')


In [16]:
#Variables globales para la simulación
global tiempo_compra_min, tiempo_compra_max, tasa_llegada, num_boletos, cifras
semilla=13
seed(semilla)

#Configuración de la simulación
num_boletos=1
tiempo_compra_min=1      # Tiempo mínimo de compra en minutos
tiempo_compra_max=5      # Tiempo máximo de compra en minutos
tasa_llegada=1/10      # Clientes llegan cada 10 minutos en promedio
num_clientes=50           # Número de clientes en la simulación
cifras=2                 # Número de decimales para los tiempos


Se escriben los parámetros iniciales para simular la compra de boletos,se determina el tiempo mínimo y máximo de compra, la tasa de llegada de los clientes y el número de boletos que se venderán.

In [23]:
global duracion_compra, tiempos_llegada, tiempos_compra, tiempos_salida
duracion_compra=0
tiempos_llegada=[]
tiempos_compra=[]
tiempos_salida=[]

#Crear entorno de simulación y recurso (fila virtual)
env=simpy.Environment()
fila_virtual=simpy.Resource(env, capacity=num_boletos)

#Proceso de venta de boletos
env.process(venta_boletos(env, fila_virtual, num_clientes))

env.run()



----El cliente número 0 ha llegado a la fila virtual en el minuto 3.34----
****El cliente 0 ha comenzado la compra en el minuto 3.34****

----El cliente número 1 ha llegado a la fila virtual en el minuto 4.77----

----El cliente número 2 ha llegado a la fila virtual en el minuto 6.89----
xxxEl cliente 0 ha terminado la compra en el minuto 8.34xxx
****El cliente 1 ha comenzado la compra en el minuto 8.34****
xxxEl cliente 1 ha terminado la compra en el minuto 11.83xxx
****El cliente 2 ha comenzado la compra en el minuto 11.83****

----El cliente número 3 ha llegado a la fila virtual en el minuto 15.46----
xxxEl cliente 2 ha terminado la compra en el minuto 16.16xxx
****El cliente 3 ha comenzado la compra en el minuto 16.16****
xxxEl cliente 3 ha terminado la compra en el minuto 18.32xxx

----El cliente número 4 ha llegado a la fila virtual en el minuto 18.8----
****El cliente 4 ha comenzado la compra en el minuto 18.8****
xxxEl cliente 4 ha terminado la compra en el minuto 20.24xxx

--